# Question: Implement LeNet-5 for MNIST Classification
Design and implement the LeNet-5 model using PyTorch

- Architectures:
    - Build a class of LeNet5 contains
        - 2 convolutional layers
            - First layer: 6 filters, kernel size 5x5, followed by a tanh activation and average pooling
            - Second layer: 16 filters, kernel size 5x5, followed by a tanh activation and average pooling
        - 3 fully connected layers;
            - First FC layer: 120 neurons
            - Second FC layer: 84 neurons.
            - Third FC layer: 10 outputs
- Data
    - Use the MNIST data set 28x28
    - Preprocess data with normalization to have a mean of 0.5 and a standard deviation of 0.5
- Training 
    - Train the model for 10 epochs using Adam optimizer with a learning rate of 0.001 and CrossEntropyLoss
    - Use a batch size of 64
- Note that for Conv layers, there is no padding which cause to reduce the dimension of the image. Also, thre is stride is 1, meaning taht the convolution capture fine-grained spatial features
            

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # Define layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5) 
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x) # 1x28x28 -> 6x24x24
        x = torch.tanh(x) # 6x24x24 -> 6x24x24
        x = self.pool(x) # 6x24x24 -> 6x12x12
        x = self.conv2(x) # 6x12x12 -> 16x8x8
        x = torch.tanh(x) # 16x8x8 -> 16x8x8
        x = self.pool(x) # 16x8x8 -> 16x4x4
        x = x.view(x.size(0), -1) # 16x4x4 -> 256
        x = self.fc1(x) # 256 -> 120
        x = self.fc2(x) # 120 -> 84
        x = self.fc3(x) # 84 -> 10
        return x

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
device = 'cpu'
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         AvgPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         AvgPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                  [-1, 120]          30,840
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.22
----------------------------------------------------------------


In [6]:
from torchviz import make_dot

# Example input for the model
x = torch.randn(1, 1, 28, 28).to(device)

# Get the graph of the model
model_graph = make_dot(model(x), params=dict(model.named_parameters()))

# Save the graph or render it
model_graph.render("images/7_1_lenet", format="png")


'images/7_1_lenet.png'

In [7]:
# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch}: Loss of {running_loss/len(train_loader):.4f}")

Epoch 0: Loss of 0.3239
Epoch 1: Loss of 0.1089
Epoch 2: Loss of 0.0814
Epoch 3: Loss of 0.0675
Epoch 4: Loss of 0.0602


In [8]:
# Test
model.eval()
correct = 0 
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 98.19
